In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[5],3c
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.058126                    1 -0.040573
0.000750        2  0.078911                    2 -0.021097
0.001052        3  0.141464                    3  0.025473
0.001476        4  0.211606                    4  0.079042
0.002070        5  0.287575                    5  0.139432
0.002904        6  0.368265                    6  0.206452
0.004074        7  0.451700                    7  0.281547
0.005714        8  0.535058                    8  0.364599
0.008015        9  0.614832                    9  0.453102
0.011243       10  0.686668                   10  0.544702
0.015771       11  0.745775                   11  0.636069
0.022122       12  0.787977                   12  0.722358
0.031031       13  0.810552                   13  0.796067
0.043528       14  0.792570                   14  0.829851
0.061057       15  0.720908                   15  0.798493
0.085645       16  0.627889                   16  0.716475
0.120136       17  0.541781                   17  0.606445
0.168516       18  0.470042                   18  0.487587
0.236378       19  0.417906                   19  0.387958
0.331549       20  0.392114                   20  0.337093
0.465100       21  0.408602                   21  0.358646
0.652400       22  0.365494                   22  0.340286
0.915100       23  0.257275                   23  0.254907
1.283650       24  0.184958                   24  0.193551
1.800600       25  0.143412                   25  0.152547
2.525700       26  0.112290                   26  0.118778
3.542800       27  0.084953                   27  0.091636
4.969550       28  0.066108                   28  0.074763
6.970850       29  0.057551                   29  0.068813
9.778100       30  0.050907                   30  0.062104
13.715850      31  0.042543                   31  0.050245
19.239350      32  0.033040                   32  0.034951
26.987250      33  0.028598                   33  0.025409
37.855300      34  0.029534                   34  0.023452
53.100050      35  0.029642                   35  0.024660
73.887500      36  0.029361                   36  0.028587
97.662500      37  0.028983                   37  0.031304
121.437500     38  0.026578                   38  0.029926
145.212500     39  0.023138                   39  0.026202
168.987500     40  0.020196                   40  0.022387
192.762500     41  0.017212                   41  0.018328
216.537500     42  0.013987                   42  0.014009
240.312500     43  0.010304                   43  0.009156
264.087500     44  0.005607                   44  0.003295
287.862500     45 -0.000864                   45 -0.004093
311.637500     46 -0.003061                   46 -0.006870
335.412500     47  0.002888                   47 -0.001207
359.187500     48  0.010780                   48  0.006789
382.962500     49  0.017491                   49  0.013740
406.737500     50  0.023930                   50  0.020311
430.512500     51  0.030396                   51  0.026790
454.287500     52  0.037012                   52  0.033363
478.062500     53  0.043814                   53  0.040292
501.837500     54  0.050766                   54  0.047454
525.612500     55  0.057778                   55  0.054703
549.387500     56  0.064692                   56  0.062107
573.162500     57  0.071330                   57  0.069448
596.937500     58  0.077488                   58  0.076458
620.712500     59  0.082980                   59  0.082804
644.487500     60  0.087672                   60  0.088381
668.262500     61  0.091507                   61  0.092968
692.037500     62  0.094548                   62  0.096782
715.812500     63  0.096931                   63  0.099940
739.587500     64  0.098874                   64  0.102640
763.362500     65  0.100648                   65  0.105133
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -16.337494  0.000000 -16.337494 -16.241849  1.898276e-09   
0.000624    2     -16.337526  0.000036 -16.337490 -16.241877  2.521253e-05   
0.000876    3     -16.337538  0.000051 -16.337488 -16.241888  3.591875e-05   
0.001229    4     -16.337556  0.000074 -16.337482 -16.241904  5.264898e-05   
0.001723    5     -16.337579  0.000110 -16.337469 -16.241925  7.875419e-05   
0.002417    6     -16.337612  0.000166 -16.337446 -16.241955  1.193512e-04   
0.003391    7     -16.337654  0.000251 -16.337403 -16.241994  1.822605e-04   
0.004757    8     -16.337711  0.000381 -16.337330 -16.242046  2.798844e-04   
0.006672    9     -16.337785  0.000576 -16.337209 -16.242114  4.310698e-04   
0.009359    10    -16.337879  0.000866 -16.337013 -16.242202  6.635616e-04   
0.013128    11    -16.337998  0.001291 -16.336706 -16.242314  1.017861e-03   
0.018415    12    -16.338141  0.001902 -16.336239 -16.242449  1.551658e-03   
0.025830    13    -16.338308  0.002760 -16.335547 -16.242607  2.343947e-03   
0.036232    14    -16.338486  0.003938 -16.334548 -16.242777  3.495444e-03   
0.050823    15    -16.338665  0.005486 -16.333179 -16.242949  5.101860e-03   
0.071291    16    -16.338842  0.007412 -16.331431 -16.243121  7.210135e-03   
0.100000    17    -16.339015  0.009720 -16.329295 -16.243287  9.812391e-03   
0.140271    18    -16.339153  0.012442 -16.326711 -16.243415  1.283389e-02   
0.196760    19    -16.339198  0.015632 -16.323566 -16.243449  1.613120e-02   
0.275997    20    -16.339033  0.019390 -16.319643 -16.243280  1.960420e-02   
0.387100    21    -16.338389  0.023907 -16.314482 -16.242643  2.340400e-02   
0.543100    22    -16.336412  0.029481 -16.306932 -16.240654  2.804289e-02   
0.761700    23    -16.332806  0.035339 -16.297467 -16.236931  3.313266e-02   
1.068500    24    -16.328615  0.040497 -16.288117 -16.232519  3.798586e-02   
1.498800    25    -16.324330  0.045641 -16.278690 -16.227928  4.326101e-02   
2.102400    26    -16.319972  0.051536 -16.268436 -16.223171  4.941233e-02   
2.949000    27    -16.315643  0.058469 -16.257175 -16.218356  5.651067e-02   
4.136600    28    -16.311799  0.066576 -16.245223 -16.213890  6.493768e-02   
5.802500    29    -16.308809  0.076631 -16.232178 -16.210082  7.588418e-02   
8.139200    30    -16.306398  0.090151 -16.216248 -16.206575  9.142723e-02   
11.417000   31    -16.304562  0.108081 -16.196482 -16.203360  1.123283e-01   
16.014700   32    -16.304237  0.130925 -16.173311 -16.201504  1.378403e-01   
22.464000   33    -16.307938  0.159868 -16.148070 -16.203873  1.669128e-01   
31.510500   34    -16.317472  0.200048 -16.117424 -16.212435  2.027066e-01   
44.200100   35    -16.332377  0.259348 -16.073030 -16.226194  2.517215e-01   
62.000000   36    -16.353986  0.343457 -16.010528 -16.246561  3.240905e-01   
85.775000   37    -16.383177  0.455339 -15.927838 -16.274856  4.329045e-01   
109.550000  38    -16.412615  0.566403 -15.846212 -16.304187  5.504054e-01   
133.325000  39    -16.444309  0.672950 -15.771359 -16.336507  6.670167e-01   
157.100000  40    -16.480793  0.774597 -15.706195 -16.374233  7.785421e-01   
180.875000  41    -16.522585  0.873269 -15.649317 -16.417924  8.852873e-01   
204.650000  42    -16.570300  0.969458 -15.600842 -16.468396  9.873819e-01   
228.425000  43    -16.624804  1.063352 -15.561451 -16.527237  1.085681e+00   
252.200000  44    -16.687507  1.155074 -15.532433 -16.596193  1.180426e+00   
275.975000  45    -16.761431  1.244790 -15.516641 -16.678384  1.271896e+00   
299.750000  46    -16.853504  1.334430 -15.519075 -16.780150  1.362135e+00   
323.525000  47    -16.964377  1.436681 -15.527696 -16.900981  1.463615e+00   
347.300000  48    -17.084967  1.565404 -15.519563 -17.031067  1.590300e+00   
371.075000  49    -17.209306  1.720105 -15.4892

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')